In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time 
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from procurement_lib import GoogleSheet

In [34]:
df_gtin = GoogleSheet("1TBAwmxAoqg6FX79d7wLu3DVtIiOb6gRfkN5fMA_tLNc")
df_gtin = df_gtin.get_as_dataframe('Hoja 1')

In [36]:
query = """
select
    competitor.competitor_name,
    site.identifier_value as site_code,
    quotation_date.full_date AS quotation_date,
    su.source_id,
    ROUND(AVG(cpp.product_selected_price),2)::float as price
from dpr_product_pricing.fact_collected_product_prices cpp
    inner join dpr_shared.dim_date quotation_date
        on cpp.dim_quotation_date = quotation_date.date_id
    inner join dpr_shared.dim_time quotation_time
        on cpp.dim_quotation_time = quotation_time.time_id
    inner join dpr_shared.dim_site site
        on cpp.dim_site = site.site_id
    inner join dpr_shared.dim_category cat
        on cpp.dim_category = cat.category_id
    inner join dpr_product_pricing.dim_product_outlier_type outlier_type
        on cpp.dim_outlier_type = outlier_type.outlier_type_id
    inner join dpr_product_pricing.dim_product_source_type source_type
        on cpp.dim_source_type = source_type.source_type_id
    inner join dpr_product_pricing.dim_product_competitor competitor
        on cpp.dim_competitor = competitor.competitor_id
    inner join dpr_product_pricing.dim_product_competitor_type competitor_type
        on(
            case
                when cpp.super_category = 'Fruver'
                    then competitor.product_competitor_type_id_fruver = competitor_type.competitor_type_id
                when cpp.super_category = 'Multicategoría'
                    then competitor.product_competitor_type_id_multicategoria = competitor_type.competitor_type_id
            end
        )
    inner join dpr_shared.dim_stock_unit su
        on cpp.dim_stock_unit = su.stock_unit_id
where quotation_date.full_date >= '2023-11-01'
    AND quotation_date.full_date < '2024-06-01'
    AND competitor.competitor_name NOT ILIKE '%cayena%'
    AND su.source_id IN {skus}
    AND site.identifier_value IN ('SPO','CWB','VCP','BHZ')
GROUP BY 1,2,3,4
""".format(skus=tuple(df_gtin.source_id.unique()))
df_zkkkkk = run_read_dwd_query(query)

In [37]:
df_zkkkkk = pd.merge(df_zkkkkk, df_gtin, left_on=['source_id','site_code'], right_on=['source_id','city'], how='left')
df_zkkkkk = df_zkkkkk.drop(['city', 'GTIN'], axis=1)
df_zkkkkk = df_zkkkkk.dropna().reset_index(drop=True)

df_zkkkkk['lifetime'] = 8

In [38]:
# Ensure dataframe is sorted by 'quotation_date'
df_zkkkkk = df_zkkkkk.sort_values(by='quotation_date')

# Generate the required rows for missing dates
new_rows = []

for (competitor, source_id), group in df_zkkkkk.groupby(['competitor_name', 'source_id']):
    group = group.sort_values(by='quotation_date')
    last_known_price = None
    last_known_date = None
    lifetime = 8
    
    for current_index in range(len(group)):
        current_date = group.iloc[current_index]['quotation_date']
        price = group.iloc[current_index]['price']
        
        # If this is not the first iteration, fill in missing dates
        if last_known_date is not None:
            days_diff = (current_date - last_known_date).days
            if days_diff > 1:
                for j in range(1, min(days_diff, lifetime + 1)):
                    new_date = last_known_date + timedelta(days=j)
                    new_row = {
                        'site_code': group.iloc[current_index]['site_code'],
                        'quotation_date': new_date,
                        'competitor_name': competitor,
                        'source_id': source_id,
                        'product_name': group.iloc[current_index]['product_name'],
                        'price': last_known_price,
                        'lifetime': lifetime - j
                    }
                    new_rows.append(new_row)
                    
                    # Stop if we reach a new datapoint date
                    if new_date + timedelta(days=1) == current_date:
                        break
        
        # Update the last known values and reset lifetime
        last_known_price = price
        last_known_date = current_date
        lifetime = 8  # Reset lifetime

    # After processing all known dates for the group, continue generating rows until lifetime reaches 0
    while lifetime > 0:
        last_known_date += timedelta(days=1)
        new_row = {
            'site_code': group.iloc[-1]['site_code'],
            'quotation_date': last_known_date,
            'competitor_name': competitor,
            'source_id': source_id,
            'product_name': group.iloc[-1]['product_name'],
            'price': last_known_price,
            'lifetime': lifetime - 1
        }
        new_rows.append(new_row)
        lifetime -= 1

# Append new rows to the dataframe
df_zkkkkk = df_zkkkkk.append(new_rows, ignore_index=True)

# Sort the final dataframe
df_zkkkkk = df_zkkkkk.sort_values(by=['competitor_name', 'source_id', 'quotation_date'])
df_zkkkkk['replica'] = df_zkkkkk['lifetime'] == 8

In [42]:
#df_zkkkkk.to_csv("info_zk_estadistic.csv",index=False)

In [16]:
df_bench = pd.read_csv("info_zk_estadistic.csv")
df_bench['quotation_date'] = pd.to_datetime(df_bench['quotation_date'])

In [21]:
df_bench.groupby('source_id').agg({'competitor_name': lambda x: x.nunique()}).reset_index().to_excel("competidores_zk.xlsx")

In [40]:
query = """
SELECT
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS quotation_date,
    cat.parent_description AS cat,
    sup.source_id,
    sup.description AS product_name,
    SUM(fs.product_quantity_x_step_unit) AS cant,
    (SUM(fs.product_price*fs.product_quantity_x_step_unit)/cant)::float AS precio,
    (SUM(fs.product_price_discount*fs.product_quantity_x_step_unit)/cant)::float AS precio_neto,
    SUM(gmv_pxq)::float AS gmv_usd


FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_stock_unit        sup  ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.gmv_pxq_local > 0
    AND sup.source_id IN {skus}
    AND cat.super_category <> 'Fruver'
    AND s.identifier_value IN ('SPO','CWB','VCP','BHZ')
    AND DATE(fs.order_submitted_date) >= '2023-11-01'
    AND DATE(fs.order_submitted_date) < '2024-06-01'

GROUP BY 1,2,3,4,5
""".format(skus=tuple(df.source_id.unique()))
df = run_read_dwd_query(query)
df['quotation_date'] = pd.to_datetime(df['quotation_date'])

In [45]:
# Función para calcular el percentil
def calcular_percentil(row, bench_df):
    source_id = row['source_id']
    quotation_date = row['quotation_date']
    precio = row['precio_neto']
    
    # Filtrar precios de referencia por source_id y quotation_date
    bench_precios = bench_df[(bench_df['source_id'] == source_id) & (bench_df['quotation_date'] == quotation_date)]['price']
    
    # Si no hay datos de referencia para ese source_id y quotation_date, devolver NaN
    if bench_precios.empty:
        return np.nan
    
    # Convertir a un array de numpy
    bench_precios = bench_precios.to_numpy(dtype=np.float64)
    
    # Manejo de valores por encima del máximo y por debajo del mínimo
    if precio < np.min(bench_precios):
        return -100*(1-(precio/np.min(bench_precios))) # Percentil mínimo
    elif precio > np.max(bench_precios):
        return 100*(precio/np.max(bench_precios)) # Percentil maximo
    
    # Calcular el percentil
    percentil = stats.percentileofscore(bench_precios, precio)
    return percentil

# Aplicar la función a cada fila de df
df['percentil'] = df.apply(calcular_percentil, axis=1, bench_df=df_bench)

In [46]:
df = df.dropna().reset_index(drop=True)

In [47]:
# df.to_csv("all_percentile_data.csv", index=False)

In [15]:
query = """
SELECT
    DATE(fs.order_submitted_date) AS quotation_date,
    sup.source_id,
    SUM(fs.gmv_pxq) AS gmv_usd,
    SUM(COALESCE(fsd.product_discount,0))/4.75 AS discount_applied,
    ((1-(SUM(inventory_p_fin*fs.product_quantity_x_step_unit)/gmv_usd))*100.00)::float AS margin,
    (margin-(100.00*discount_applied/gmv_usd))::float AS net_margin
    
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_stock_unit        sup  ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id
LEFT JOIN dpr_cross_business.obt_cross_business m ON m.stock_unit_id = sup.stock_unit_id AND DATE(m.close_date-1) = DATE(fs.order_submitted_date)

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND sup.source_id IN {skus}
    AND cat.super_category = 'Multicategoría'
    AND m.inventory_p_fin > 0
    AND fs.gmv_pxq_local > 0
    AND s.identifier_value IN ('SPO','CWB','VCP','BHZ')
    AND fs.order_submitted_date >= '2023-11-01'
    AND fs.order_submitted_date < '2024-06-01'
GROUP BY 1,2
""".format(skus=tuple(df.source_id.unique()))

dfm = run_read_dwd_query(query)
dfm['quotation_date'] = pd.to_datetime(dfm['quotation_date'])

In [17]:
df4 = pd.merge(df, dfm[['quotation_date', 'source_id','margin','net_margin']], left_on=['quotation_date', 'source_id'], right_on=['quotation_date', 'source_id'], how='left')

In [18]:
df4['cash_margin'] = df4.gmv_usd * df4.margin / 100.00
df4['net_cash_margin'] = df4.gmv_usd * df4.net_margin / 100.00

In [20]:
# df4.to_csv("all_percentile_data_margin.csv", index=False)

In [2]:
df = pd.read_csv("all_percentile_data_margin.csv")
df['quotation_date'] = pd.to_datetime(df['quotation_date'])

In [3]:
# Calculate weighted average percentil
weighted_avg_percentil = (
    df.assign(weighted_percentil=df['percentil'] * df['gmv_usd'])
    .groupby(['region_code','cat','source_id','product_name'])
    .agg(weighted_sum=('weighted_percentil', 'sum'), gmv_usd_sum=('gmv_usd', 'sum'))
    .assign(weighted_avg_percentil=lambda x: x['weighted_sum'] / x['gmv_usd_sum'])
    .reset_index()
    [['region_code','cat','source_id','product_name','gmv_usd_sum','weighted_avg_percentil']]
)

In [12]:
# weighted_avg_percentil.to_excel("percentilecity.xlsx")

In [4]:
df_B = pd.read_csv("grafica_B_zk.csv")
df_B['quotation_date'] = pd.to_datetime(df_B['quotation_date'])

In [5]:
# Merge the DataFrames
merged_df = pd.merge(df_B, df, left_on=['quotation_date', 'source_id'], right_on=['quotation_date', 'source_id'], how='left')
merged_df['product_name'] = merged_df['region_code'] + ' || ' + merged_df['product_name_x']

In [6]:
merged_df.to_excel("info_compete_zk.xlsx",index=False)

In [22]:
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact

# Function to plot data for a single product with additional metrics
def plot_product_prices_with_complement(product_name):
    product_data = merged_df[merged_df['product_name'] == product_name]

    fig = go.Figure()

    # Plot the summary statistics
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['min'], mode='lines', name='Min'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['q1'], mode='lines', name='Q1'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['p10'], mode='lines', name='P10'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['median'], mode='lines', name='Median'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['q3'], mode='lines', name='Q3'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['max'], mode='lines', name='Max'))
    
    # Plot the additional values with connectgaps=True
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['precio'], mode='markers+lines', name='Sale Price', line=dict(dash='dash'), marker=dict(symbol='circle'), connectgaps=True))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['precio_neto'], mode='markers+lines', name='NET Sale Price', line=dict(dash='dash'), marker=dict(symbol='circle'), connectgaps=True))

    fig.update_layout(
        title=f'Price Summary and Additional Metrics for Product ID: {product_name}',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Metrics',
        height=800,
        template='plotly_white'
    )

    fig.show()

# Get unique product IDs
product_names = pd.Series(merged_df.product_name.unique()).sort_values()

# Create dropdown widget
dropdown = widgets.Dropdown(options=product_names, description='Product Name:')

# Interact function to update plot based on dropdown selection
interact(plot_product_prices_with_complement, product_name=dropdown)

interactive(children=(Dropdown(description='Product Name:', options=('BHZ || Achocolatado em Pó Nescau 2.0 Sac…

<function __main__.plot_product_prices_with_complement(product_name)>

In [7]:
df['product_name'] = df.region_code + ' || ' + df.product_name

In [8]:
df2 = df[['quotation_date','source_id','product_name','gmv_usd','percentil', 'cash_margin','net_cash_margin']].copy()

In [9]:
# Assuming df2 is your DataFrame
# Ensure 'quotation_date' is in datetime format
df2['quotation_date'] = pd.to_datetime(df2['quotation_date'])

# Create a new column for the start of the week
df2['week_start'] = df2['quotation_date'] - pd.to_timedelta(df2['quotation_date'].dt.weekday, unit='d')

# Group by the start of the week, source_id, and product_name
weekly_group = df2.groupby(['week_start', 'source_id', 'product_name'])

# Calculate the sum of 'gmv_usd' for each group
weekly_gmv_sum = weekly_group['gmv_usd'].sum().reset_index(name='weekly_gmv_sum')

# Calculate the sum of 'cash_margin' for each group
weekly_cm_sum = weekly_group['cash_margin'].sum().reset_index(name='weekly_cm_sum')

# Calculate the sum of 'net_cash_margin' for each group
weekly_ncm_sum = weekly_group['net_cash_margin'].sum().reset_index(name='weekly_ncm_sum')

# Calculate the weighted percentile for each group
weekly_weighted_percentil = weekly_group.apply(
    lambda x: (x['percentil'] * x['gmv_usd']).sum() / x['gmv_usd'].sum()
).reset_index(name='weighted_percentil')

# Merge the results
weekly_summary = pd.merge(weekly_gmv_sum, weekly_weighted_percentil, on=['week_start', 'source_id', 'product_name'])
weekly_summary = pd.merge(weekly_summary, weekly_cm_sum, on=['week_start', 'source_id', 'product_name'])
weekly_summary = pd.merge(weekly_summary, weekly_ncm_sum, on=['week_start', 'source_id', 'product_name'])

In [10]:
weekly_summary

,week_start,source_id,product_name,weekly_gmv_sum,weighted_percentil,weekly_cm_sum,weekly_ncm_sum
0,2023-10-30,72318,SPO || Feijão Preto Camil 1Kg Pacote de 1kg Un...,6072.41619,47.345912,-44.319862,-44.319862
1,2023-10-30,72329,SPO || Feijão Preto Kicaldo 1Kg Pacote de 1kg ...,3461.75462,37.828775,104.305740,104.305740
2,2023-10-30,72488,SPO || Feijão Carioca Camil 1Kg Pacote de 1kg ...,15953.34433,71.882425,422.672407,-683.835936
3,2023-10-30,72497,SPO || Feijão Carioca Kicaldo 1Kg Pacote de 1k...,12110.66844,78.022707,577.079537,169.094374
4,2023-10-30,73456,SPO || Açúcar Refinado União 1Kg Pacote de 1kg...,5903.36729,12.758196,188.688225,89.218627
...,...,...,...,...,...,...,...
46018,2024-05-27,652383,CWB || Sassami de Frango Envelopado CX 20Kg - ...,467.03158,31.453300,44.004643,44.004643
46019,2024-05-27,652431,VCP || Sabão em pó Triex Flowers 800g Pacote d...,15.83998,210.548757,2.469580,2.469580
46020,2024-05-27,652441,SPO || Sassami de Frango Envelopado CX 20Kg - ...,1546.23151,15.826519,198.683360,198.683360
46021,2024-05-27,652453,BHZ || Filé de Peito de Frango Envelopado CX 2...,5443.74724,56.713513,637.179024,401.056087


In [13]:
# Function to plot data for a single product with additional metrics
def plot_product_prices_with_complement(product_name):
    product_data = weekly_summary[weekly_summary['product_name'] == product_name]

    fig = go.Figure()

    # Plot the summary statistics
    fig.add_trace(go.Scatter(x=product_data['week_start'], y=product_data['weighted_percentil'], mode='markers+lines', name='Percentil', line=dict(dash='dash'), marker=dict(symbol='circle'), connectgaps=True))
    # Plot the GMV on the secondary y-axis
    fig.add_trace(go.Scatter(x=product_data['week_start'], y=product_data['weekly_gmv_sum'], mode='lines', name='GMV', yaxis='y2'))
    fig.add_trace(go.Scatter(x=product_data['week_start'], y=product_data['weekly_cm_sum'], mode='lines', name='CM', yaxis='y2'))
    fig.add_trace(go.Scatter(x=product_data['week_start'], y=product_data['weekly_ncm_sum'], mode='lines', name='NCM', yaxis='y2'))
    
    # Update layout to add secondary y-axis
    fig.update_layout(
        title=f'Price Summary and Additional Metrics for Product ID: {product_name}',
        xaxis_title='Date',
        yaxis_title='Percentile',
        yaxis2=dict(
            title='GMV',
            overlaying='y',
            side='right'
        ),
        legend_title='Metrics',
        height=800,
        template='plotly_white'
    )

    fig.show()

# Get unique product IDs
product_names = pd.Series(weekly_summary.product_name.unique()).sort_values()

# Create dropdown widget
dropdown = widgets.Dropdown(options=product_names, description='Product Name:')

# Interact function to update plot based on dropdown selection
interact(plot_product_prices_with_complement, product_name=dropdown)

interactive(children=(Dropdown(description='Product Name:', options=('BHZ || Achocolatado em Pó Nescau 2.0 Sac…

<function __main__.plot_product_prices_with_complement(product_name)>